In [36]:
from bs4 import BeautifulSoup
import requests
import re
import os
import json

#url = "https://www.artsper.com/fr/oeuvres-d-art-contemporain/peinture/2300112/evasion"
url = "https://www.artsper.com/fr/oeuvres-d-art-contemporain/peinture/2272663/discogolf"
response = requests.get(url, timeout=10)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

In [37]:
title = soup.find("title")
print(title)

<title>▷ Discogolf par Ralph Resch, 2024 | Peinture | Artsper</title>


In [38]:
title_text = title.text.strip()
year_part = title_text.split(",")[-1].split("|")[0].strip()
print(year_part)

2024


In [39]:
name = title_text.split(",")[0].split('par')[0]
name = re.sub(r"[^a-zA-Z0-9\s]", "", name).strip()
print(name)
#print('Evasion')

Discogolf


In [40]:
artist_name = title_text.split(",")[0].split('par')[1].strip()
print(artist_name)

Ralph Resch


In [41]:
target_element = soup.select_one('div.top-information__price span.price.price__current.typography--bold').text.strip()
print(target_element)

1 050 €


In [42]:

# Find all 'a' tags with the specific data-infos pattern
# We'll look for 'a' tags that have a 'data-infos' attribute containing 'peinture'
techniques = []
for a_tag in soup.find_all('a', class_='pointer'):
    if a_tag.get('data-infos') and 'peinturestcdsq' in a_tag['data-infos']:
        # Check if the text does not contain "Peinture"
        if "Peinture" not in a_tag.text:
          techniques.append(a_tag.text.strip().rstrip(',')) #rstrip removes the trailing commas

# Print the extracted techniques
if techniques:
    print(", ".join(techniques))  # Nicely formatted output
else:
    print("No techniques found.")

huile, acrylique, ciment


In [43]:
description_divs = soup.find_all('div', class_='about__block__item__description')

# Use a counter to track which dimension we're on
count = 1

for div in description_divs:
    # Check if the div contains a span with class 'measure--cm'
    span = div.find('span', class_='measure--cm')
    if span:
        # If it contains the span, extract the dimensions
        dimensions_text = span.get_text(strip=True)

        # Assign the appropriate label based on the count
        if count == 1:
            label = "Dimensions"
        elif count == 2:
            label = "Framed work dimensions"
        else:
            label = "Unknown dimension"  # Handle cases with more than 2

        print(f"{label}: {dimensions_text}")

        # Increment the counter
        count += 1

Dimensions: 116 x 89 x 2 cm
Framed work dimensions: 117 x 90 x 3 cm


In [44]:
tags = []

# Find all 'a' and 'span' tags with the class 'button'
for tag in soup.find_all(['a', 'span'], class_='button'):
    # Check if the tag has the necessary attributes to be a tag
    if tag.get('data-category') == 'Interaction' and tag.get('data-label') == 'Tags':
        # Find the h3 tag within the current tag and extract its text
        h3_tag = tag.find('h3', class_='typography__text')
        if h3_tag:
            tag_text = h3_tag.get_text(strip=True)
            tags.append(tag_text)

# Print the extracted tags
if tags:
    print(", ".join(tags))  # Nicely formatted output
else:
    print("No tags found.")

Abstrait, Composition, Œuvres d'art couleurs, vert, Artistes inspirés par Rothko, Artistes Peintres, Artistes Français


In [45]:
results = {}

# Find all divs with the class 'about__block__item'
about_items = soup.find_all('div', class_='about__block__item')

for item in about_items:
    # Find the title (p tag with class 'about__block__item__title')
    title_tag = item.find('p', class_='about__block__item__title')
    if title_tag:
        title = title_tag.get_text(strip=True)

        # --- Handle Support specifically ---
        if "Support" in title:
            description_tag = item.find('div', class_='about__block__item__description')
            if description_tag:
                a_tag = description_tag.find('a', class_='pointer', attrs={'data-infos': lambda x: x and 'support-toile' in x})
                if a_tag:
                    results[title] = a_tag.get_text(strip=True) # Store the text of <a>

        # --- Handle Encadrement and other potential sections ---
        elif "Encadrement" in title: #Use elif, to look at others if not support
            description_tag = item.find('div', class_='about__block__item__description')
            if description_tag:
                results[title] = description_tag.get_text(strip=True)  #store text

# Access the values using the keys
print(results.get("Support", "Support Not Found"))
print(results.get("Encadrement", "Encadrement Not Found"))

Peinture sur toile sur chassis
Cadre noir


In [46]:
img_tag = soup.find('img', id='img-viar')

if img_tag:
    # Get the image URL from the 'data-src' attribute
    image_url = img_tag.get('data-src')

    if image_url:
        print(f"Image URL: {image_url}")  # Print the URL
    else:
        print("Image URL not found in 'data-src' attribute.")
else:
    print("Image tag with id='img-viar' not found.")

Image URL: https://media.artsper.com/artwork/2272663_1_l.jpg


In [47]:
filename = os.path.basename(image_url)
# If the filename doesn't have an extension, add one (e.g., .jpg)
if not os.path.splitext(filename)[1]:
    filename += ".jpg"  # Or .png, .gif, etc., if you know the type

# --- 2.  Download the image ---
try:
    response = requests.get(image_url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    # --- 3. Save the image to a file ---
    with open(filename, 'wb') as file:  # 'wb' mode for writing binary data
        for chunk in response.iter_content(chunk_size=8192): #iter_content to prevent large files to take a large amount of memory
            file.write(chunk)

    print(f"Image downloaded and saved as: {filename}")

except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
except OSError as e: # Catch OSError to hand bad paths, invalid characters, etc
    print(f"Error saving image: {e}")

Image downloaded and saved as: 2272663_1_l.jpg
